In [1]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [2]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압출 풀기
with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
# 테스트 이미지 데이터 압출 풀기
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [3]:
import torch # 파이토치 

torch.manual_seed(0)

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
device

device(type='cpu')

In [6]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(labels, 
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=10)

In [7]:
print('훈련 데이터 개수:', len(train))
print('검증 데이터 개수:', len(valid))

훈련 데이터 개수: 15750
검증 데이터 개수: 1750


In [8]:
import cv2
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    # 초기화 메서드(생성자) ---①
    def __init__(self, df, img_dir = './', transform=None):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출 ---②
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터 세트 크기 반환 메서드 ---③
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드 ---④
    def __getitem__(self, idx):
        img_path = self.img_dir + self.df.iloc[idx, 0] # 이미지 파일 경로 
        image = cv2.imread(img_path) # 이미지 파일 읽기 
        label = self.df.iloc[idx, 1] # 이미지 레이블(타깃 값)
        # 이미지 변환
        if self.transform:
            image = self.transform(image)
        return image, label

In [9]:
import torchvision.transforms as transforms # 이미지 변환을 위한 모듈

transform = transforms.ToTensor()

In [10]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

In [11]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=128, 
                          shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=64, 
                          shuffle=False)

In [12]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

class Model(nn.Module):
    # 신경망 계층 정의 ---①
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
        # 첫 번째 합성곱 계층 ---②
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                               kernel_size=3, padding=2) 
        # 두 번째 합성곱 계층 ---③
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, 
                               kernel_size=3, padding=2) 
        # 최대 풀링 계층 ---④
        self.max_pool = nn.MaxPool2d(kernel_size=2) 
        # 평균 풀링 계층 ---⑤
        self.avg_pool = nn.AvgPool2d(kernel_size=2) 
        # 전결합 계층 ---⑥
        self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)
        
    # 순전파 출력 정의 ---⑦
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4) # 평탄화
        x = self.fc(x)
        return x

In [13]:
model = Model().to(device)

model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

In [14]:
# 손실함수
criterion = nn.CrossEntropyLoss()

In [15]:
# 옵티마이저
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [16]:
import math

math.ceil(len(train) / 128)

124

In [17]:
len(loader_train)

124

In [18]:
iters = len(loader_train) # 반복 횟수
epochs = 25 # 총 에폭

# 총 25에폭만큼 훈련 ---①
for epoch in range(epochs):
    # 미니배치 크기만큼 데이터를 추출하는 작업을 '반복 횟수'만큼 반복 ---②
    for i, (images, labels) in enumerate(loader_train):
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 ---③
        images = images.to(device)
        labels = labels.to(device)

        # 옵티마이저 내 기울기 초기화 ---④
        optimizer.zero_grad()
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산 ---⑤
        outputs = model(images)
        # 기존에 정의한 손실함수를 활용해 outputs와 labels의 손실값 계산 ---⑥
        loss = criterion(outputs, labels)
        # 오차 역전파 수행 ---⑦
        loss.backward()
        # 가중치 갱신 ---⑧
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'에폭: [{epoch+1}/{epochs}], 훈련 스텝: [{i+1}/{iters}], 손실값: {loss.item():.4f}')

에폭: [1/25], 훈련 스텝: [100/124], 손실값: 0.5834
에폭: [2/25], 훈련 스텝: [100/124], 손실값: 0.5793
에폭: [3/25], 훈련 스텝: [100/124], 손실값: 0.4803
에폭: [4/25], 훈련 스텝: [100/124], 손실값: 0.4559
에폭: [5/25], 훈련 스텝: [100/124], 손실값: 0.3587
에폭: [6/25], 훈련 스텝: [100/124], 손실값: 0.3317
에폭: [7/25], 훈련 스텝: [100/124], 손실값: 0.2644
에폭: [8/25], 훈련 스텝: [100/124], 손실값: 0.2713
에폭: [9/25], 훈련 스텝: [100/124], 손실값: 0.2338
에폭: [10/25], 훈련 스텝: [100/124], 손실값: 0.1700
에폭: [11/25], 훈련 스텝: [100/124], 손실값: 0.2169
에폭: [12/25], 훈련 스텝: [100/124], 손실값: 0.2276
에폭: [13/25], 훈련 스텝: [100/124], 손실값: 0.2217
에폭: [14/25], 훈련 스텝: [100/124], 손실값: 0.1505
에폭: [15/25], 훈련 스텝: [100/124], 손실값: 0.1525
에폭: [16/25], 훈련 스텝: [100/124], 손실값: 0.2738
에폭: [17/25], 훈련 스텝: [100/124], 손실값: 0.2214
에폭: [18/25], 훈련 스텝: [100/124], 손실값: 0.1654
에폭: [19/25], 훈련 스텝: [100/124], 손실값: 0.1677
에폭: [20/25], 훈련 스텝: [100/124], 손실값: 0.1870
에폭: [21/25], 훈련 스텝: [100/124], 손실값: 0.1547
에폭: [22/25], 훈련 스텝: [100/124], 손실값: 0.1791
에폭: [23/25], 훈련 스텝: [100/124], 손실값: 0.1519
에폭: [24/25], 훈련 스텝: 

In [19]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

roc_auc = 0 # ROC AUC 초기화

model.eval() # 모델을 평가 상태로 설정 ---①

with torch.no_grad(): # 기울기 계산 비활성 ---②
    for images, labels in loader_valid:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측 확률값 ---③
        true = labels.cpu() # 실제 값 ---④
        # 검증 데이터 ROC AUC 점수 계산 ---⑤
        roc_auc += roc_auc_score(true, preds) / len(loader_valid)

    print(f'검증 데이터 ROC AUC : {roc_auc}')

검증 데이터 ROC AUC : 0.9901276463302303


In [20]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

In [21]:
model.eval() # 모델을 평가 상태로 설정

preds = [] # 타깃 예측 값 저장용 변수 초기화 ---①

with torch.no_grad(): # 기울기 계산 비활성
    for images, _ in loader_test:
        # 이미지 데이터 미니배치를 장비에 할당
        images = images.to(device)
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 타깃 값이 1일 확률(예측 값) ---②
        preds_temp = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        # preds에 preds_temp 이어붙이기 ---③
        preds.extend(preds_temp)

In [22]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [23]:
submission

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,0.948558
1,0017242f54ececa4512b4d7937d1e21e.jpg,0.999325
2,001ee6d8564003107853118ab87df407.jpg,0.028275
3,002e175c3c1e060769475f52182583d0.jpg,0.092357
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.832344
...,...,...
3995,ffaafd0c9f2f0e73172848463bc2e523.jpg,0.984172
3996,ffae37344310a1549162493237d25d3f.jpg,0.999856
3997,ffbd469c56873d064326204aac546e0d.jpg,0.997739
3998,ffcb76b7d47f29ece11c751e5f763f52.jpg,0.969420


In [24]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')